In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()
import warnings
warnings.filterwarnings('ignore')
from sqlalchemy import create_engine

## Flights

In [2]:
flight = pd.read_csv("flights.csv")

In [3]:
flight.shape

(5819079, 31)

In [4]:
flight.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#make pandas dataframe column headers all lowercase
flight.columns = map(str.lower, flight.columns)

In [6]:
#select columns
flight_df = flight[["year", "month", "day","departure_time", "arrival_time", "origin_airport", 
                    "destination_airport", "departure_delay", "arrival_delay", "airline",
                   "air_system_delay", "security_delay", "airline_delay", "late_aircraft_delay", "weather_delay"]]

In [7]:
flight_df["flight_id"] = range(1, len(flight_df)+1)

In [8]:
flight_df.head()

,year,month,day,departure_time,arrival_time,origin_airport,destination_airport,departure_delay,arrival_delay,airline,air_system_delay,security_delay,airline_delay,late_aircraft_delay,weather_delay,flight_id
0,2015,1,1,2354.0,408.0,ANC,SEA,-11.0,-22.0,AS,NaN,NaN,NaN,NaN,NaN,1
1,2015,1,1,2.0,741.0,LAX,PBI,-8.0,-9.0,AA,NaN,NaN,NaN,NaN,NaN,2
2,2015,1,1,18.0,811.0,SFO,CLT,-2.0,5.0,US,NaN,NaN,NaN,NaN,NaN,3
3,2015,1,1,15.0,756.0,LAX,MIA,-5.0,-9.0,AA,NaN,NaN,NaN,NaN,NaN,4
4,2015,1,1,24.0,259.0,SEA,ANC,-1.0,-21.0,AS,NaN,NaN,NaN,NaN,NaN,5


In [9]:
#rename the columns
flight_df.columns = ["departure_year", "departure_month", "departure_day", "departure_time", 
                     "arrival_time", "origin_airport", "destination_airport", "departure_delay",
                     "arrival_delay", "airline", "air_system_delay", "security_delay", 
                     "airline_delay", "late_aircraft_delay", "weather_delay", "flight_id", ]

#reorder the columns
flight_df = flight_df[["flight_id","departure_year", "departure_month", "departure_day", "departure_time", 
             "arrival_time", "origin_airport", "destination_airport", "departure_delay",
             "arrival_delay", "airline", "air_system_delay", "security_delay", "airline_delay", "late_aircraft_delay", "weather_delay"]]

In [10]:
flight_df

,flight_id,departure_year,departure_month,departure_day,departure_time,arrival_time,origin_airport,destination_airport,departure_delay,arrival_delay,airline,air_system_delay,security_delay,airline_delay,late_aircraft_delay,weather_delay
0,1,2015,1,1,2354.0,408.0,ANC,SEA,-11.0,-22.0,AS,NaN,NaN,NaN,NaN,NaN
1,2,2015,1,1,2.0,741.0,LAX,PBI,-8.0,-9.0,AA,NaN,NaN,NaN,NaN,NaN
2,3,2015,1,1,18.0,811.0,SFO,CLT,-2.0,5.0,US,NaN,NaN,NaN,NaN,NaN
3,4,2015,1,1,15.0,756.0,LAX,MIA,-5.0,-9.0,AA,NaN,NaN,NaN,NaN,NaN
4,5,2015,1,1,24.0,259.0,SEA,ANC,-1.0,-21.0,AS,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5819074,5819075,2015,12,31,2355.0,753.0,LAX,BOS,-4.0,-26.0,B6,NaN,NaN,NaN,NaN,NaN
5819075,5819076,2015,12,31,2355.0,430.0,JFK,PSE,-4.0,-16.0,B6,NaN,NaN,NaN,NaN,NaN
5819076,5819077,2015,12,31,2350.0,432.0,JFK,SJU,-9.0,-8.0,B6,NaN,NaN,NaN,NaN,NaN
5819077,5819078,2015,12,31,2353.0,330.0,MCO,SJU,-6.0,-10.0,B6,NaN,NaN,NaN,NaN,NaN


## Airlines

In [11]:
#read data
airline = pd.read_csv("airlines.csv")

#create an ID column
airline["airline_id"] = range(1, len(airline)+1)

#make pandas dataframe column headers all lowercase
airline = airline[["airline_id", "AIRLINE", "IATA_CODE"]]

#rename columns
airline.columns = ["airline_id", "airline_name", "IATA_airline_code"]

In [12]:
#merge dfs
flight_airline_df = pd.merge(flight_df, 
                            airline,
                            left_on = "airline",
                            right_on= 'IATA_airline_code')

#remove columns
flight_airline_df.drop(['airline_name', 'airline', 'airline_name', 'IATA_airline_code'], axis=1, inplace=True)

In [13]:
flight_airline_df.head()

,flight_id,departure_year,departure_month,departure_day,departure_time,arrival_time,origin_airport,destination_airport,departure_delay,arrival_delay,air_system_delay,security_delay,airline_delay,late_aircraft_delay,weather_delay,airline_id
0,1,2015,1,1,2354.0,408.0,ANC,SEA,-11.0,-22.0,NaN,NaN,NaN,NaN,NaN,7
1,5,2015,1,1,24.0,259.0,SEA,ANC,-1.0,-21.0,NaN,NaN,NaN,NaN,NaN,7
2,16,2015,1,1,41.0,455.0,ANC,SEA,-4.0,-14.0,NaN,NaN,NaN,NaN,NaN,7
3,19,2015,1,1,46.0,507.0,ANC,PDX,-4.0,-18.0,NaN,NaN,NaN,NaN,NaN,7
4,27,2015,1,1,107.0,545.0,FAI,SEA,-8.0,-3.0,NaN,NaN,NaN,NaN,NaN,7


## Airports

In [14]:
airport = pd.read_csv("airports.csv")
airport["airport_id"] = range(1, len(airport)+1)
airport.columns = map(str.lower, airport.columns)
airport = airport[["airport_id", "airport", "iata_code", "city", "state", "country", "latitude", "longitude"]]
airport.rename(columns={'iata_code': 'IATA_airport_code',
                        'airport':'airport_name'}, inplace=True)
airport.dropna(subset=["latitude", "longitude"], how='all', inplace=True)

In [15]:
airport.head()

,airport_id,airport_name,IATA_airport_code,city,state,country,latitude,longitude
0,1,Lehigh Valley International Airport,ABE,Allentown,PA,USA,40.65236,-75.44040
1,2,Abilene Regional Airport,ABI,Abilene,TX,USA,32.41132,-99.68190
2,3,Albuquerque International Sunport,ABQ,Albuquerque,NM,USA,35.04022,-106.60919
3,4,Aberdeen Regional Airport,ABR,Aberdeen,SD,USA,45.44906,-98.42183
4,5,Southwest Georgia Regional Airport,ABY,Albany,GA,USA,31.53552,-84.19447


In [16]:
#handle departure airport
flight_origin_airport_df = pd.merge(flight_airline_df, 
                                    airport,
                                    left_on = "origin_airport",
                                    right_on= 'IATA_airport_code')

flight_origin_airport_df.drop(['country','state', 'city', 'airport_name', 'IATA_airport_code',"latitude", 
                               "longitude", "origin_airport"], axis=1, inplace=True)

flight_origin_airport_df.rename(columns={'airport_id': 'departure_airport'}, inplace=True)

In [17]:
#handle destination airport
flight_des_airport_df = pd.merge(flight_origin_airport_df, 
                                airport,
                                left_on = "destination_airport",
                                right_on= 'IATA_airport_code')

flight_des_airport_df.drop(['country','state', 'city', 'airport_name', 'IATA_airport_code', 
                            "latitude", "longitude", "destination_airport"], axis=1, inplace=True)

flight_des_airport_df.rename(columns={'airport_id': 'destination_airport'}, inplace=True)


In [18]:
flight_des_airport_df = flight_des_airport_df[["flight_id", "departure_year", "departure_month", "departure_day", "departure_time",
                                               "arrival_time", "departure_delay", "arrival_delay", "airline_id", "departure_airport", 
                                               "destination_airport", "air_system_delay", "security_delay", 
                                               "airline_delay", "late_aircraft_delay", "weather_delay"]]

In [19]:
new_flight_df = flight_des_airport_df.dropna(subset=["departure_time", "arrival_time", "departure_delay", "arrival_delay"])

In [20]:
new_flight_df.head()

,flight_id,departure_year,departure_month,departure_day,departure_time,arrival_time,departure_delay,arrival_delay,airline_id,departure_airport,destination_airport,air_system_delay,security_delay,airline_delay,late_aircraft_delay,weather_delay
0,1,2015,1,1,2354.0,408.0,-11.0,-22.0,7,18,278,NaN,NaN,NaN,NaN,NaN
1,16,2015,1,1,41.0,455.0,-4.0,-14.0,7,18,278,NaN,NaN,NaN,NaN,NaN
3,39,2015,1,1,140.0,558.0,-15.0,-35.0,7,18,278,NaN,NaN,NaN,NaN,NaN
4,48,2015,1,1,209.0,628.0,-11.0,-12.0,7,18,278,NaN,NaN,NaN,NaN,NaN
5,62,2015,1,1,457.0,916.0,-8.0,-14.0,7,18,278,NaN,NaN,NaN,NaN,NaN


## Load

In [ ]:
engine = create_engine('postgresql://postgres:postgres/flight_db')

In [ ]:
connection = engine.connect()

In [ ]:
engine.table_names()

In [ ]:
airline.to_sql(name='airline', con=engine, if_exists='append', index=False)

In [ ]:
airport.to_sql(name='airport', con=engine, if_exists='append', index=False)